# SCRAPING

### 1º Import all the libraries.

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re

### 2º Import the new information with URL.

In [2]:
# https://es.wikipedia.org/wiki/Anexo:Comunidades_y_ciudades_aut%C3%B3nomas_de_Espa%C3%B1a

In [3]:
url = input("Introduce the URL: ")

Introduce the URL: https://es.wikipedia.org/wiki/Anexo:Comunidades_y_ciudades_aut%C3%B3nomas_de_Espa%C3%B1a


In [4]:
response = requests.get(url)

In [5]:
response.status_code

200

In [6]:
soup = BeautifulSoup(response.content)
#soup

### 3º Extract the  `Comunidades Autónomas` and `Number of Poblation`.

In [7]:
soup.select("td b a")[5]

<a href="/wiki/Comunidad_de_Madrid" title="Comunidad de Madrid">Comunidad de Madrid</a>

In [8]:
len(soup.select("td b a"))

38

In [9]:
names = soup.select("td b a")

In [10]:
def extract_names(names):
    return [n.text for i,n in enumerate(names) if i % 2 != 0]

In [11]:
lst_name = extract_names(names)
lst_name

['Andalucía',
 'Cataluña',
 'Comunidad de Madrid',
 'Comunidad Valenciana',
 'Galicia',
 'Castilla y León',
 'País Vasco',
 'Canarias',
 'Castilla-La Mancha',
 'Región de Murcia',
 'Aragón',
 'Islas Baleares',
 'Extremadura',
 'Principado de Asturias',
 'Navarra',
 'Cantabria',
 'La Rioja',
 'Melilla',
 'Ceuta']

In [12]:
num_pob = soup.find_all("span", attrs = {"style":"display:none"})

In [13]:
len(num_pob)

60

In [14]:
def extract_numbers(num_pob):
    
    list_num_pob = list(enumerate(num_pob))
    list_poblation = []
    
    for n in list_num_pob:
    
        list_range = list(range(0,55,3))

        if n[0] in list_range:

            list_poblation.append(n[1])

    return list_poblation

In [15]:
list_poblation = extract_numbers(num_pob)
#list_poblation

In [16]:
def text_numbers(list_poblation):
    return [n.text for n in list_poblation]

In [17]:
new_list_poblation = text_numbers(list_poblation)
#new_list_poblation

In [18]:
def string_numbers(new_list_poblation):
    st_poblation = [str(n) for n in new_list_poblation]
    return str(st_poblation)

In [19]:
list_str_poblation = string_numbers(new_list_poblation)
#list_str_poblation

In [20]:
def regex_numbers(pattern,list_str_poblation):
    return re.findall(pattern,list_str_poblation)

In [21]:
pattern = r'\b\d{6,8}'
poblation = regex_numbers(pattern,list_str_poblation)
#poblation

In [22]:
def int_numbers(poblation):
    return [int(p) for p in poblation]

In [23]:
poblation_int = int_numbers(poblation)
#poblation_int

### 4º Create a Dictionary with the infromation. 

In [24]:
dictionary = dict(zip(lst_name,poblation_int))

In [61]:
#dictionary

# MERGE THE SCRAPING AND DATASET.

In [27]:
df = pd.read_csv("../data/data_clean.csv")

In [59]:
#df

In [32]:
def pob(x):
    for k,v in dictionary.items():
        if x == k:
            return v

In [34]:
df["Poblation"] = df["Comunidad Autónoma"].apply(pob)

In [39]:
df.head(3)

,Month,Comunidad Autónoma,Total Paro Registrado,Hombre edad < 25,Hombre edad 25-45,Hombre edad >=45,Mujer edad < 25,Mujer edad 25-45,Mujer edad >= 45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior,Poblation
0,Apr,Andalucía,978297,43593,190621,194878,42170,261692,245343,71597,61299,101074,657397,86930,8464411
1,Apr,Aragón,79214,4269,14028,15627,3851,19993,21446,3591,8990,5444,55588,5601,1329391
2,Apr,Principado de Asturias,83793,3047,16267,18868,2723,20084,22804,1433,7211,7657,60588,6904,1018784


In [42]:
df["% Total Paro Registrado"] = df["Total Paro Registrado"] / df["Poblation"]

In [46]:
df["% Hombre edad < 25"] = df["Hombre edad < 25"] / df["Poblation"]

In [48]:
df["% Hombre edad 25-45"] = df["Hombre edad 25-45"] / df["Poblation"]

In [49]:
df["% Hombre edad >=45"] = df["Hombre edad >=45"] / df["Poblation"]

In [50]:
df["% Mujer edad < 25"] = df["Mujer edad < 25"] / df["Poblation"]

In [51]:
df["% Mujer edad 25-45"] = df["Mujer edad 25-45"] / df["Poblation"]

In [52]:
df["% Mujer edad >= 45"] = df["Mujer edad >= 45"] / df["Poblation"]

In [53]:
df["% Paro Agricultura"] = df["Paro Agricultura"] / df["Poblation"]

In [54]:
df["% Paro Industria"] = df["Paro Industria"] / df["Poblation"]

In [55]:
df["% Paro Construcción"] = df["Paro Construcción"] / df["Poblation"]

In [56]:
df["% Paro Servicios"] = df["Paro Servicios"] / df["Poblation"]

In [57]:
df["% Paro Sin empleo Anterior"] = df["Paro Sin empleo Anterior"] / df["Poblation"]

In [58]:
df.head(3)

,Month,Comunidad Autónoma,Total Paro Registrado,Hombre edad < 25,Hombre edad 25-45,Hombre edad >=45,Mujer edad < 25,Mujer edad 25-45,Mujer edad >= 45,Paro Agricultura,...,% Hombre edad 25-45,% Hombre edad >=45,% Mujer edad < 25,% Mujer edad 25-45,% Mujer edad >= 45,% Paro Agricultura,% Paro Industria,% Paro Construcción,% Paro Servicios,% Paro Sin empleo Anterior
0,Apr,Andalucía,978297,43593,190621,194878,42170,261692,245343,71597,...,0.022520,0.023023,0.004982,0.030917,0.028985,0.008459,0.007242,0.011941,0.077666,0.010270
1,Apr,Aragón,79214,4269,14028,15627,3851,19993,21446,3591,...,0.010552,0.011755,0.002897,0.015039,0.016132,0.002701,0.006762,0.004095,0.041815,0.004213
2,Apr,Principado de Asturias,83793,3047,16267,18868,2723,20084,22804,1433,...,0.015967,0.018520,0.002673,0.019714,0.022384,0.001407,0.007078,0.007516,0.059471,0.006777


In [60]:
df.to_csv("data_merge.csv",index=False)